# LSTM Implementation

In [1]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(y):
    return y * (1 - y)

def dtanh(x):
    return 1 - x**2

In [2]:
class LSTM:
  def __init__(self,input_size,hidden_size):
    self.input_size =input_size
    self.hidden_size = hidden_size
    concat_size = input_size + hidden_size
    self.Wf = np.random.randn(hidden_size, concat_size) * 0.1 #forget
    self.bf = np.zeros((hidden_size, 1))
    self.Wi = np.random.randn(hidden_size, concat_size) * 0.1 #input
    self.bi = np.zeros((hidden_size, 1))
    self.Wg = np.random.randn(hidden_size, concat_size) * 0.1 #candidate
    self.bg = np.zeros((hidden_size, 1))
    self.Wo = np.random.randn(hidden_size, concat_size) * 0.1 #output
    self.bo = np.zeros((hidden_size, 1))
  def forward(self,inputs):
    h_prev = np.zeros((self.hidden_size,1)) # short term
    c_prev = np.zeros((self.hidden_size,1)) # long term
    self.cache = []
    for x_t in inputs:
        x_t = x_t.reshape(-1, 1)
        z = np.vstack((h_prev, x_t))

        f = sigmoid(self.Wf @ z + self.bf) # range 0 to 1 ; 0 : forget ,1 : keep
        i = sigmoid(self.Wi @ z + self.bi) # How much to write. 0: no write ,1 :full write
        g = np.tanh(self.Wg @ z + self.bg) # candidate values to add .-1 ,1 or 0 or in between what to write
        # i * g  means: "these are the new things to remember"
        c = f * c_prev + i * g # f -> 0 forget c_prev completely,1 means keep c_prev as it is .
        o = sigmoid(self.Wo @ z + self.bo) # How much memory you show
        h = o * np.tanh(c)
        #c can have large values.
        #tanh(c) bounds it to [-1, 1] — just like we discussed earlier.
        #Without this, h could be too large → unstable.
        self.cache.append((f, i, g, o, c, h, c_prev, h_prev, z)) #You store everything needed to compute gradients later.
        h_prev, c_prev = h, c
        return h
  def backward(self, dL_dh, lr=0.01):
        dWf = np.zeros_like(self.Wf)
        dWi = np.zeros_like(self.Wi)
        dWg = np.zeros_like(self.Wg)
        dWo = np.zeros_like(self.Wo)
        dbf = np.zeros_like(self.bf)
        dbi = np.zeros_like(self.bi)
        dbg = np.zeros_like(self.bg)
        dbo = np.zeros_like(self.bo)

        dh_next = dL_dh # gradient of loss w.r.t. last hidden state given
        dc_next = np.zeros((self.hidden_size, 1))

        for t in reversed(range(len(self.cache))):
            f, i, g, o, c, h, c_prev, h_prev, z = self.cache[t]

            do = dh_next * np.tanh(c)
            dco = dh_next * o * dtanh(np.tanh(c)) + dc_next

            df = dco * c_prev
            di = dco * g
            dg = dco * i
            dc_prev = dco * f

            dzf = df * dsigmoid(f)
            dzi = di * dsigmoid(i)
            dzg = dg * dtanh(g)
            dzo = do * dsigmoid(o)

            dWf += dzf @ z.T
            dWi += dzi @ z.T
            dWg += dzg @ z.T
            dWo += dzo @ z.T
            dbf += dzf
            dbi += dzi
            dbg += dzg
            dbo += dzo

            dz = (
                self.Wf.T @ dzf +
                self.Wi.T @ dzi +
                self.Wg.T @ dzg +
                self.Wo.T @ dzo
            )

            dh_next = dz[:self.hidden_size, :]
            dc_next = dc_prev

        # Update parameters
        for param, dparam in zip(
            [self.Wf, self.Wi, self.Wg, self.Wo, self.bf, self.bi, self.bg, self.bo],
            [dWf, dWi, dWg, dWo, dbf, dbi, dbg, dbo]
        ):
            param -= lr * dparam